In [ ]:
# import the hourly weather data
# add type of day information: include the weekday/weekends, holiday

In [55]:
import pandas as pd
import numpy as np
import datetime

In [30]:
weather = pd.read_csv('../../../Data/Data processing/Weather data/Dataset/weather_hourly_201801-202105.csv')
flow = pd.read_csv('../../New_flow/GD1351_R_predict_2019-11-20.csv')

In [8]:
display(flow)

,varFlow,timeStamp(t+12),time(t+12),newFlow(t+12)
0,62.0,2019-11-20 12:00:00,12.00,113.0
1,62.0,2019-11-20 12:15:00,12.25,113.0
2,62.0,2019-11-20 12:30:00,12.50,113.0
3,62.0,2019-11-20 12:45:00,12.75,113.0
4,62.0,2019-11-20 13:00:00,13.00,117.0
...,...,...,...,...
72331,31.0,2021-12-12 22:45:00,22.75,40.0
72332,53.0,2021-12-12 23:00:00,23.00,48.0
72333,66.0,2021-12-12 23:15:00,23.25,53.0
72334,44.0,2021-12-12 23:30:00,23.50,35.0


## 1. Hourly weather data

### 1.1 Reconstruct the weather data

In [31]:
#create a timeStamp column for weather data
weather_new = weather[['dates', 'time', 'humidity', 'visibility','weather_type','temp', 'pressure', 'wind_speed','dew_point']]
weather_new['timeStamp'] = weather_new['dates'].str.split('Z').str[0] + ' ' + weather_new['time'].astype('int').astype(str).str.zfill(2)

E:\program\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
weather_new

,dates,time,humidity,visibility,weather_type,temp,pressure,wind_speed,dew_point,timeStamp
0,2018-01-22Z,21.0,88.8,15000,8,6.4,1008,14,4.7,2018-01-22 21
1,2018-01-22Z,22.0,92.0,13000,7,6.3,1008,9,5.1,2018-01-22 22
2,2018-01-22Z,23.0,89.5,18000,7,6.8,1007,11,5.2,2018-01-22 23
3,2018-01-23Z,0.0,90.0,23000,8,6.4,1007,8,4.9,2018-01-23 00
4,2018-01-23Z,1.0,97.3,3500,11,6.0,1006,8,5.6,2018-01-23 01
...,...,...,...,...,...,...,...,...,...,...
27803,2021-05-07Z,18.0,50.8,75000,1,9.5,1012,10,-0.1,2021-05-07 18
27804,2021-05-07Z,19.0,54.5,70000,1,9.0,1013,8,0.4,2021-05-07 19
27805,2021-05-07Z,20.0,61.5,70000,1,6.6,1012,5,-0.2,2021-05-07 20
27806,2021-05-07Z,21.0,67.8,70000,2,4.9,1012,7,-0.5,2021-05-07 21


### 1.2 Reconstruct the flow data

In [41]:
#create a timeStamp column for flow data
fullTime = pd.read_csv('../../New_flow/GD1351_R_new.csv')[['timeStamp']].sort_values(by=['timeStamp']).reset_index(drop=True)
#select the data from 2019-11-20
start_index = fullTime.loc[fullTime.timeStamp == '2019-11-20 00:00:00'].index.item()
end_index = fullTime.loc[fullTime.timeStamp == '2021-05-07 23:45:00'].index.item()+1
partTime = fullTime.iloc[start_index:end_index].reset_index(drop=True)
partTime

,timeStamp
0,2019-11-20 00:00:00
1,2019-11-20 00:15:00
2,2019-11-20 00:30:00
3,2019-11-20 00:45:00
4,2019-11-20 01:00:00
...,...
51355,2021-05-07 22:45:00
51356,2021-05-07 23:00:00
51357,2021-05-07 23:15:00
51358,2021-05-07 23:30:00


In [46]:
# slice the flow data from 2019-11-20 to 2021-05-07
flow_new = pd.concat([flow, partTime], axis=1).dropna()
flow_new['timeStamp'] = flow_new['timeStamp'].str.split(':').str[0]
flow_new

,varFlow,timeStamp(t+12),time(t+12),newFlow(t+12),timeStamp
0,62.0,2019-11-20 12:00:00,12.00,113.0,2019-11-20 00
1,62.0,2019-11-20 12:15:00,12.25,113.0,2019-11-20 00
2,62.0,2019-11-20 12:30:00,12.50,113.0,2019-11-20 00
3,62.0,2019-11-20 12:45:00,12.75,113.0,2019-11-20 00
4,62.0,2019-11-20 13:00:00,13.00,117.0,2019-11-20 01
...,...,...,...,...,...
51355,22.0,2021-05-08 10:45:00,10.75,58.0,2021-05-07 22
51356,22.0,2021-05-08 11:00:00,11.00,53.0,2021-05-07 23
51357,31.0,2021-05-08 11:15:00,11.25,70.0,2021-05-07 23
51358,22.0,2021-05-08 11:30:00,11.50,92.0,2021-05-07 23


### 1.3 Merge the weather data to flow

In [79]:
# merge the weather data to flow
flow_weather = pd.merge(flow_new, weather_new[['humidity', 'visibility','weather_type','temp', 'pressure', 'wind_speed','dew_point','timeStamp']], how="left", on="timeStamp")
flow_weather[['humidity', 'visibility','weather_type','temp', 'pressure', 'wind_speed','dew_point']] = flow_weather[['humidity', 'visibility','weather_type','temp', 'pressure', 'wind_speed','dew_point']].astype("float")

In [87]:
flow_weather[flow_weather.isnull().any(axis=1)]

,varFlow,timeStamp(t+12),time(t+12),newFlow(t+12),timeStamp,humidity,visibility,weather_type,temp,pressure,wind_speed,dew_point


### 1.4 Interpolate the weather data

In [86]:
flow_weather = flow_weather.interpolate(method='nearest').ffill().bfill()

,varFlow,timeStamp(t+12),time(t+12),newFlow(t+12),timeStamp,humidity,visibility,weather_type,temp,pressure,wind_speed,dew_point
16124,40.0,2020-05-06 11:00:00,11.00,57.0,2020-05-05 23,73.8,55000.0,0.0,7.8,1025.0,3.0,3.5
16125,9.0,2020-05-06 11:15:00,11.25,57.0,2020-05-05 23,73.8,55000.0,0.0,7.8,1025.0,3.0,3.5
16126,22.0,2020-05-06 11:30:00,11.50,66.0,2020-05-05 23,73.8,55000.0,0.0,7.8,1025.0,3.0,3.5
16127,22.0,2020-05-06 11:45:00,11.75,75.0,2020-05-05 23,73.8,55000.0,0.0,7.8,1025.0,3.0,3.5
16128,26.0,2020-05-06 12:00:00,12.00,75.0,2020-05-06 00,73.8,55000.0,0.0,7.8,1025.0,3.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...
51355,22.0,2021-05-08 10:45:00,10.75,58.0,2021-05-07 22,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4
51356,22.0,2021-05-08 11:00:00,11.00,53.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4
51357,31.0,2021-05-08 11:15:00,11.25,70.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4
51358,22.0,2021-05-08 11:30:00,11.50,92.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4


In [88]:
flow_weather

,varFlow,timeStamp(t+12),time(t+12),newFlow(t+12),timeStamp,humidity,visibility,weather_type,temp,pressure,wind_speed,dew_point
0,62.0,2019-11-20 12:00:00,12.00,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1
1,62.0,2019-11-20 12:15:00,12.25,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1
2,62.0,2019-11-20 12:30:00,12.50,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1
3,62.0,2019-11-20 12:45:00,12.75,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1
4,62.0,2019-11-20 13:00:00,13.00,117.0,2019-11-20 01,97.1,3300.0,12.0,3.0,1009.0,2.0,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...
51355,22.0,2021-05-08 10:45:00,10.75,58.0,2021-05-07 22,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4
51356,22.0,2021-05-08 11:00:00,11.00,53.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4
51357,31.0,2021-05-08 11:15:00,11.25,70.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4
51358,22.0,2021-05-08 11:30:00,11.50,92.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4


## 2. Create the type of day

In [96]:
flow_weather['date'] = flow_weather['timeStamp(t+12)'].str.split(' ').str[0]

,varFlow,timeStamp(t+12),time(t+12),newFlow(t+12),timeStamp,humidity,visibility,weather_type,temp,pressure,wind_speed,dew_point,date
0,62.0,2019-11-20 12:00:00,12.00,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2019-11-20
1,62.0,2019-11-20 12:15:00,12.25,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2019-11-20
2,62.0,2019-11-20 12:30:00,12.50,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2019-11-20
3,62.0,2019-11-20 12:45:00,12.75,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2019-11-20
4,62.0,2019-11-20 13:00:00,13.00,117.0,2019-11-20 01,97.1,3300.0,12.0,3.0,1009.0,2.0,2.6,2019-11-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51355,22.0,2021-05-08 10:45:00,10.75,58.0,2021-05-07 22,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,2021-05-08
51356,22.0,2021-05-08 11:00:00,11.00,53.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,2021-05-08
51357,31.0,2021-05-08 11:15:00,11.25,70.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,2021-05-08
51358,22.0,2021-05-08 11:30:00,11.50,92.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,2021-05-08


In [108]:
flow_weather['dayType(t+12)'] = 0
for i in range(len(flow_weather)):
    date_list = flow_weather['date'][i].split('-')
    daytype = datetime.datetime(int(date_list[0]), int(date_list[1]), int(date_list[2])).weekday()
    flow_weather['dayType(t+12)'][i] = daytype
flow_weather

E:\program\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,varFlow,timeStamp(t+12),time(t+12),newFlow(t+12),timeStamp,humidity,visibility,weather_type,temp,pressure,wind_speed,dew_point,date,dayType(t+12)
0,62.0,2019-11-20 12:00:00,12.00,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2019-11-20,2
1,62.0,2019-11-20 12:15:00,12.25,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2019-11-20,2
2,62.0,2019-11-20 12:30:00,12.50,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2019-11-20,2
3,62.0,2019-11-20 12:45:00,12.75,113.0,2019-11-20 00,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2019-11-20,2
4,62.0,2019-11-20 13:00:00,13.00,117.0,2019-11-20 01,97.1,3300.0,12.0,3.0,1009.0,2.0,2.6,2019-11-20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51355,22.0,2021-05-08 10:45:00,10.75,58.0,2021-05-07 22,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,2021-05-08,5
51356,22.0,2021-05-08 11:00:00,11.00,53.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,2021-05-08,5
51357,31.0,2021-05-08 11:15:00,11.25,70.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,2021-05-08,5
51358,22.0,2021-05-08 11:30:00,11.50,92.0,2021-05-07 23,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,2021-05-08,5


## 3. Create the data ready for training

In [111]:
flow_weather = flow_weather.drop(columns=['timeStamp', 'date'])
flow_weather

,varFlow,timeStamp(t+12),time(t+12),newFlow(t+12),humidity,visibility,weather_type,temp,pressure,wind_speed,dew_point,dayType(t+12)
0,62.0,2019-11-20 12:00:00,12.00,113.0,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2
1,62.0,2019-11-20 12:15:00,12.25,113.0,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2
2,62.0,2019-11-20 12:30:00,12.50,113.0,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2
3,62.0,2019-11-20 12:45:00,12.75,113.0,97.8,4300.0,15.0,3.4,1009.0,5.0,3.1,2
4,62.0,2019-11-20 13:00:00,13.00,117.0,97.1,3300.0,12.0,3.0,1009.0,2.0,2.6,2
...,...,...,...,...,...,...,...,...,...,...,...,...
51355,22.0,2021-05-08 10:45:00,10.75,58.0,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,5
51356,22.0,2021-05-08 11:00:00,11.00,53.0,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,5
51357,31.0,2021-05-08 11:15:00,11.25,70.0,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,5
51358,22.0,2021-05-08 11:30:00,11.50,92.0,78.6,50000.0,0.0,2.9,1012.0,1.0,-0.4,5


In [112]:
#flow_weather.to_csv('../../New_flow/GD1351_R_weather&day_2019-11-20_2021-05-07.csv', index = False)